<a href="https://colab.research.google.com/github/rociavl/SEEG_automatic_segmentation/blob/main/3D_filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3D Filters


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls '/content/drive/MyDrive/TFG 💪🧠/Code/DATA/'


P1  P2	P3  P4	P5  P6	P7  P8


In [3]:
!pip install SimpleITK
!pip install pynrrd
!pip install scikit-image
!pip install scipy
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 14.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import scipy.ndimage as ndi
import SimpleITK as sitk  # For handling NRRD files
from scipy.ndimage import gaussian_filter, median_filter, laplace, sobel
import logging
from skimage.filters import frangi
from pathlib import Path
import os
from skimage.measure import regionprops
import logging
import nrrd
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter, median_filter, sobel
from pathlib import Path
import os
from skimage.measure import label, regionprops
from scipy.ndimage import gaussian_filter
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


In [5]:
# Read NRRD file
def read_nrrd_file(file_path):
    """Reads an NRRD file and returns the image as a numpy array and SimpleITK image."""
    try:
        image = sitk.ReadImage(file_path)
        image_array = sitk.GetArrayFromImage(image)  # Convert to numpy array
        print(f"✅ Successfully read NRRD file: {file_path}")
        return image_array, image
    except Exception as e:
        logging.error(f"❌ Error reading NRRD file: {str(e)}")
        return None, None


# Save NRRD file
def save_nrrd_file(image_array, image, output_file):
    """Saves a processed volume as an NRRD file using SimpleITK."""
    try:
        output_image = sitk.GetImageFromArray(image_array)
        output_image.SetSpacing(image.GetSpacing())
        output_image.SetOrigin(image.GetOrigin())
        sitk.WriteImage(output_image, output_file)
        print(f"✅ Saved NRRD file to {output_file}")
    except Exception as e:
        logging.error(f"❌ Error saving NRRD file: {str(e)}")

In [6]:
def remove_large_objects(volume, size_threshold):
    """
    Removes objects that are larger than a given size threshold.

    Parameters:
        volume (numpy.ndarray): The 3D image (volume).
        size_threshold (int): The size threshold for removing large objects.

    Returns:
        numpy.ndarray: The volume with large objects removed.
    """
    try:
        # Label connected components
        labeled_volume, num_features = label(volume > 0)  # Only keep foreground

        # Get the properties of the labeled regions
        regions = regionprops(labeled_volume)

        # Create an empty volume to hold the filtered results
        filtered_volume = np.zeros_like(volume)

        # Loop through the regions and keep only those below the size threshold
        for region in regions:
            if region.area <= size_threshold:
                for coord in region.coords:
                    filtered_volume[tuple(coord)] = volume[tuple(coord)]  # Keep small objects

        print(f"✅ Removed {num_features - len([r for r in regions if r.area <= size_threshold])} large objects")
        return filtered_volume

    except Exception as e:
        logging.error(f"❌ Error removing large objects: {str(e)}")
        return volume


# Vesselness Filter (Custom Implementation)
def vesselness_filter(image, sigma=1.0, alpha=0.5, beta=0.5):
    """
    Applies custom vesselness filter for enhancing vessels.

    Parameters:
        image (numpy.ndarray): The 3D image (volume) to be filtered.
        sigma (float): The standard deviation of the Gaussian filter used in Hessian computation.
        alpha (float): The sensitivity parameter for vesselness calculation.
        beta (float): The sensitivity parameter for vesselness calculation.

    Returns:
        numpy.ndarray: The vesselness-enhanced image.
    """
    print(f"Applying vesselness filter with sigma={sigma}, alpha={alpha}, beta={beta}")
    try:
        # Compute Hessian manually
        hessian_image = np.array([gaussian_filter(image, sigma=sigma, order=(2, 0, 0)),
                                  gaussian_filter(image, sigma=sigma, order=(0, 2, 0)),
                                  gaussian_filter(image, sigma=sigma, order=(0, 0, 2))])

        # Compute eigenvalues of the Hessian matrix
        eigvals = np.linalg.eigvalsh(hessian_image)

        # Sort eigenvalues by absolute value
        eigvals = np.sort(np.abs(eigvals), axis=0)

        # Compute vesselness measure
        vesselness = np.zeros_like(image)
        lambda1, lambda2, lambda3 = eigvals[0], eigvals[1], eigvals[2]
        vesselness = (1 - np.exp(-lambda2**2 / (2 * alpha**2))) * np.exp(-lambda3**2 / (2 * beta**2))

        # Normalize vesselness
        vesselness = (vesselness - vesselness.min()) / (vesselness.max() - vesselness.min())

        return vesselness

    except Exception as e:
        logging.error(f"Error applying vesselness filter: {str(e)}")
        return None


# Sobel filter for edge detection (CPU)
def sobel_filter(image):
    """Applies Sobel edge detection filter on the CPU."""
    print("Applying Sobel edge detection filter")
    try:
        edges = np.sqrt(sobel(image, axis=0)**2 + sobel(image, axis=1)**2 + sobel(image, axis=2)**2)
        return edges
    except Exception as e:
        logging.error(f"Error applying Sobel filter: {str(e)}")
        return None


# Gamma correction (CPU)
def gamma_correction(image, gamma=1.0):
    """Applies Gamma correction to the image."""
    print(f"Applying Gamma correction with gamma={gamma}")
    try:
        image = np.power(image, gamma)
        return image
    except Exception as e:
        logging.error(f"Error applying gamma correction: {str(e)}")
        return None

def hessian_filter(image, sigma=1.0):
    """Apply Hessian-based filtering for blob detection."""
    # Compute the Hessian matrix at each voxel
    grad_xx = gaussian_filter(image, sigma=sigma, order=(2, 0, 0))
    grad_yy = gaussian_filter(image, sigma=sigma, order=(0, 2, 0))
    grad_zz = gaussian_filter(image, sigma=sigma, order=(0, 0, 2))
    grad_xy = gaussian_filter(image, sigma=sigma, order=(1, 1, 0))
    grad_xz = gaussian_filter(image, sigma=sigma, order=(1, 0, 1))
    grad_yz = gaussian_filter(image, sigma=sigma, order=(0, 1, 1))

    # Construct the Hessian matrix
    hessian = np.array([[[grad_xx, grad_xy, grad_xz],
                         [grad_xy, grad_yy, grad_yz],
                         [grad_xz, grad_yz, grad_zz]]])

    # Use eigenvalues of the Hessian matrix for blob detection
    eigenvalues = np.linalg.eigvals(hessian)
    return eigenvalues

def frangi_filter(image, sigma=1.0, beta=0.5):
    """Apply Frangi filter for enhancing tubular structures."""
    # Compute Hessian
    grad_xx = gaussian_filter(image, sigma=sigma, order=(2, 0, 0))
    grad_yy = gaussian_filter(image, sigma=sigma, order=(0, 2, 0))
    grad_zz = gaussian_filter(image, sigma=sigma, order=(0, 0, 2))

    # Compute the eigenvalues
    eigvals = np.linalg.eigvals(np.array([[[grad_xx, grad_yy, grad_zz]]]))

    # Compute the vesselness score
    vesselness = np.exp(-eigvals[1] ** 2 / (2 * beta ** 2)) * np.exp(-eigvals[2] ** 2 / (2 * beta ** 2))

    return vesselness

def anisotropic_diffusion(image, num_iter=10, kappa=50, gamma=0.1):
    """Apply Anisotropic Diffusion (Perona-Malik) on a 3D image."""
    image = np.asarray(image)

    for _ in range(num_iter):
        # Calculate gradients
        gradient_x = np.diff(image, axis=0, append=image[-1:])
        gradient_y = np.diff(image, axis=1, append=image[-1:])
        gradient_z = np.diff(image, axis=2, append=image[-1:])

        # Compute the conductance (edge-preserving term)
        c_x = np.exp(-(gradient_x / kappa) ** 2)
        c_y = np.exp(-(gradient_y / kappa) ** 2)
        c_z = np.exp(-(gradient_z / kappa) ** 2)

        # Update the image using the conductance values
        image[1:, :, :] += gamma * c_x[1:, :, :] * (image[:-1, :, :] - image[1:, :, :])
        image[:, 1:, :] += gamma * c_y[:, 1:, :] * (image[:, :-1, :] - image[:, 1:, :])
        image[:, :, 1:] += gamma * c_z[:, :, 1:] * (image[:, :, :-1] - image[:, :, 1:])

    return image

In [7]:
# Apply 3D filter method (CPU)
def apply_3d_filter(volume, method, params=None):
    """Applies various 3D filters using CPU-based methods."""
    if params is None:
        params = {}

    try:
        print(f"Applying {method} filter with params: {params}")

        # Apply filters using NumPy (CPU-based)
        if method == 'gaussian':
            sigma = params.get('sigma', 1.5)
            result = gaussian_filter(volume, sigma=sigma)
        elif method == 'median':
            size = params.get('size', 1)
            result = median_filter(volume, size=size)
        elif method == 'laplacian':
            result = np.gradient(volume)  # A simple approach to Laplacian
        elif method == 'vesselness':
            result = vesselness_filter(volume, sigma=params.get('sigma', 1.0), alpha=params.get('alpha', 0.5), beta=params.get('beta', 0.5))
        elif method == 'sobel':
            result = sobel_filter(volume)
        elif method == 'gamma_correction':
            result = gamma_correction(volume, gamma=params.get('gamma', 1.0))
        else:
            logging.error(f"Unknown filter method: {method}")
            return None

        # Normalize output
        result = (result - result.min()) / (result.max() - result.min())
        print(f"{method} filter applied and normalized successfully")
        return result

    except Exception as e:
        logging.error(f"Error applying {method} filter: {str(e)}")
        return None


# Process volume with cascading filters using CPU

In [8]:
def process_volume_3d(input_volume_path, output_dir=None, methods=None):
    """Processes the volume with multiple filters in sequence."""
    try:
        # Load the input volume
        image_array, image = read_nrrd_file(input_volume_path)

        if image_array is None or image is None:
            print("❌ Failed to load input volume.")
            return

        # Apply filters in sequence
        enhanced_volumes = {}
        for method, params in methods:
            print(f"Processing method: {method}")
            enhanced_image = apply_3d_filter(image_array, method, params)

            if enhanced_image is None:
                print(f"❌ Skipping method {method} due to error.")
                continue

            enhanced_volumes[method] = enhanced_image

            # Save the enhanced volume as an NRRD file
            if output_dir:
                output_path = Path(output_dir)
                output_path.mkdir(parents=True, exist_ok=True)

                # Construct the output file path
                output_file = output_path / f"Enhanced_{method}_{os.path.basename(input_volume_path)}.nrrd"
                save_nrrd_file(enhanced_image, image, str(output_file))

        return enhanced_volumes

    except Exception as e:
        logging.error(f"Error during volume processing: {str(e)}")

In [9]:
from pathlib import Path

output_dir = '/content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D'
output_path = Path(output_dir)
output_path.mkdir(parents=True, exist_ok=True)  # Ensure directory is created
if output_path.exists():
    print(f"✅ Output directory exists or was created: {output_dir}")
else:
    print(f"❌ Failed to create output directory: {output_dir}")


✅ Output directory exists or was created: /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D


In [12]:
# Example
input_volume_path ='/content/Filtered_og_ctp.3D.nrrd'
methods = [
    ('gaussian', {'sigma': 0.5}),
    ('gamma_correction', {'gamma': 4}),
    ('vesselness', {'sigma': 1.0, 'alpha': 0.5, 'beta': 0.5}),
    ('sobel', {}),
]

process_volume_3d(input_volume_path, output_dir=output_dir, methods=methods)

✅ Successfully read NRRD file: /content/Filtered_og_ctp.3D.nrrd
Processing method: gaussian
Applying gaussian filter with params: {'sigma': 0.5}
gaussian filter applied and normalized successfully
✅ Saved NRRD file to /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D/Enhanced_gaussian_Filtered_og_ctp.3D.nrrd.nrrd
Processing method: gamma_correction
Applying gamma_correction filter with params: {'gamma': 4}
Applying Gamma correction with gamma=4
gamma_correction filter applied and normalized successfully
✅ Saved NRRD file to /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D/Enhanced_gamma_correction_Filtered_og_ctp.3D.nrrd.nrrd
Processing method: vesselness
Applying vesselness filter with params: {'sigma': 1.0, 'alpha': 0.5, 'beta': 0.5}
Applying vesselness filter with sigma=1.0, alpha=0.5, beta=0.5
vesselness filter applied and normalized successfully
✅ Saved NRRD file to /content/drive/MyDrive/TFG 💪🧠/Code/DATA/P1/P1/Enhance_ctp_3D/Enhanced_vesselness_Filtered_og_

{'gaussian': array([[[0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         ...,
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206]],
 
        [[0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.23917206, 0.23917206],
         ...,
         [0.23917206, 0.23917206, 0.23917206, ..., 0.23917206,
          0.

Labeling and centroides



In [ ]:
def detect_shape_and_coordinates(volume, shape_type='sphere', threshold=0.5):
    """
    Detects regions in a 3D volume based on their shape and returns their coordinates and centroids.
    """
    try:
        # Step 1: Apply threshold to the volume
        binary_volume = volume > threshold

        # Step 2: Label the connected components in the binary volume
        labeled_volume, num_labels = label(binary_volume)

        # Step 3: Analyze each region (connected component)
        shapes_data = []

        for region in regionprops(labeled_volume):
            # Calculate the centroid of the region
            centroid = region.centroid

            # Get the coordinates of the region
            region_coords = region.coords

            # Calculate the bounding box and aspect ratio (for shape detection)
            minr, minc, minz, maxr, maxc, maxz = region.bbox
            length = maxr - minr
            width = maxc - minc
            height = maxz - minz

            # Aspect ratio for shape classification (simple criteria for shapes)
            aspect_ratio = max(length, width, height) / min(length, width, height)

            # Check shape types based on aspect ratio and volume
            if shape_type == 'sphere':
                # For spheres, aspect ratio should be close to 1
                if 0.9 <= aspect_ratio <= 1.1:
                    shapes_data.append({
                        'type': 'sphere',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })
            elif shape_type == 'square':
                # For squares (in 2D) or cubes in 3D, aspect ratio should be close to 1
                if 0.9 <= aspect_ratio <= 1.1 and region.area > 100:  # Only considering large regions
                    shapes_data.append({
                        'type': 'square',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })
            elif shape_type == 'rectangle':
                # For rectangles (or rectangular prisms in 3D), aspect ratio should be greater than 1
                if aspect_ratio > 1.2:
                    shapes_data.append({
                        'type': 'rectangle',
                        'centroid': centroid,
                        'coordinates': region_coords.tolist(),
                        'volume': region.area
                    })

        # Step 4: Convert results into a DataFrame for easy export
        shapes_df = pd.DataFrame(shapes_data)
        return shapes_df

    except Exception as e:
        logging.error(f"Error detecting shapes: {str(e)}")
        return None


# Example usage
volume = np.random.random((100, 100, 100))  # Replace with your 3D volume (e.g., from NRRD file)
shapes_df = detect_shape_and_coordinates(volume, shape_type='sphere', threshold=0.5)

if shapes_df is not None:
    # Save to CSV or print the results
    shapes_df.to_csv('shapes_data.csv', index=False)
    print(shapes_df)